<a href="https://colab.research.google.com/github/Taimaaaa/Pipeline-ColumnTransformer-Practice/blob/main/Pipelines_%26_ColumnTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocessing Data Before Building ML Model


##Question we want to answer:
How well can the "rating" of cereal be predicted using the following features?
  + mfr. type. calories. protein, fat. fiber, sugars. shelf

In [5]:
#important imports
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn import set_config
set_config(transform_output='pandas')

In [1]:
#mounting my drive to extract the dataset from there
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#reading from file path
fpath="/content/drive/MyDrive/AXSOSACADEMY/AXSOSACADEMY/02-IntroML/Week05/Data/cereal-kaggle-crawford-modified - sheet 1.csv"
df=pd.read_csv(fpath)

In [7]:
#dataset overview
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      77 non-null     object 
 1   mfr       75 non-null     object 
 2   type      77 non-null     object 
 3   calories  72 non-null     float64
 4   protein   77 non-null     int64  
 5   fat       70 non-null     float64
 6   sodium    77 non-null     int64  
 7   fiber     71 non-null     float64
 8   carbo     77 non-null     float64
 9   sugars    71 non-null     float64
 10  potass    77 non-null     int64  
 11  vitamins  77 non-null     int64  
 12  shelf     75 non-null     object 
 13  weight    77 non-null     float64
 14  cups      77 non-null     float64
 15  rating    77 non-null     float64
dtypes: float64(8), int64(4), object(4)
memory usage: 9.8+ KB


,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,NaN,4,1.0,130,10.0,5.0,6.0,280,25,top,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120.0,3,5.0,15,2.0,8.0,8.0,135,0,top,1.0,1.00,33.983679
2,All-Bran,K,C,70.0,4,1.0,260,9.0,7.0,5.0,320,25,top,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50.0,4,0.0,140,14.0,8.0,0.0,330,25,top,1.0,0.50,93.704912
4,Almond Delight,R,C,NaN,2,2.0,200,1.0,14.0,8.0,-1,25,NaN,1.0,0.75,34.384843


## Pipelines

How well can the calories be predicted based on the Manufacturer, cereal type, grams of fat, grams of sugars, and weight in ounces per one serving of the cereal?

Define features (X) and target (y).

In [8]:
# Define features (X) and target (y).
# X should include every feature except: "name."
# y should be 'Rating of cereal'

target = 'rating'
X = df.drop(columns=[target , 'rating'])
y = df[target]

Train test split the data to prepare for machine learning.

In [9]:
# Train test split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

Identify each feature as numerical, ordinal, or nominal.

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 30 to 51
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      57 non-null     object 
 1   mfr       55 non-null     object 
 2   type      57 non-null     object 
 3   calories  57 non-null     float64
 4   protein   57 non-null     int64  
 5   fat       52 non-null     float64
 6   sodium    57 non-null     int64  
 7   fiber     52 non-null     float64
 8   carbo     57 non-null     float64
 9   sugars    52 non-null     float64
 10  potass    57 non-null     int64  
 11  vitamins  57 non-null     int64  
 12  shelf     57 non-null     object 
 13  weight    57 non-null     float64
 14  cups      57 non-null     float64
dtypes: float64(7), int64(4), object(4)
memory usage: 7.1+ KB


In [11]:
# Checking object columns
X_train.select_dtypes('object')

,name,mfr,type,shelf
30,Golden Crisp,P,C,bottom
40,Kix,G,C,middle
39,Just Right Fruit & Nut,K,C,top
16,Corn Flakes,K,C,bottom
65,Shredded Wheat spoon size,N,C,bottom
54,Puffed Rice,Q,C,top
31,Golden Grahams,G,C,middle
7,Basic 4,G,C,top
62,Rice Krispies,K,C,bottom
44,Muesli Raisins; Dates; & Almonds,R,C,top


In [12]:
X_train['mfr'].value_counts(dropna=False)

,count
mfr,
K,18
G,17
P,6
Q,5
R,5
N,3
NaN,2
A,1


In [13]:
X_train['type'].value_counts(dropna=False)

,count
type,
C,54
H,3


In [14]:
X_train['shelf'].value_counts(dropna=False)

,count
shelf,
top,26
bottom,17
middle,14


## Features:

- Ordinal: Display shelf
- Categorical: Manufacturer and type.
- The remaining features are numeric.

I will create 3 pipelines (one for numeric, ordinal, and categorical features).

For the numeric features/pipeline I will:

- Impute null values using SImpleImputer using the ‘mean’ strategy.
- Scale the data with StandardScaler.

For categorical (nominal) pipeline:

- Impute null values using SimpleImputer using the ‘constant’ strategy with a fill value of "MISSING."

- Use OneHotEncoder to encode the features.
  - Be sure to include the arguments: sparse_output=False AND handle_unknown='ignore' when creating the OneHotEncoder.


For the ordinal pipeline:
- Impute null values using SimpleImputer using the "most_frequent" strategy.
- Use OrdinalEncoder to encode the "shelf" column.
- Scale the ordinal features using StandardScaler

In [15]:
# Prepare ordinal pipeline
ord_cols = ['shelf']

# Impute Most common
impute_common = SimpleImputer(strategy='most_frequent')

# Ordinal Encoder
shelf_order = ['bottom','middle','top']
ord_encoder = OrdinalEncoder(categories=[shelf_order])

# Scaler
scaler = StandardScaler()

# make a pipeline
ord_pipe = make_pipeline(impute_common, ord_encoder, scaler)
ord_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['bottom', 'middle', 'top']])),
                ('standardscaler', StandardScaler())])

In [16]:
# Prepare Categorical pipeline
cat_cols = X_train.select_dtypes('object').drop(columns=ord_cols).columns

impute_missing = SimpleImputer(strategy='constant',fill_value='MISSING')
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_pipe = make_pipeline(impute_missing, ohe_encoder)
ohe_pipe

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='MISSING', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

In [17]:
# Prepare numeric piepline
num_cols = X_train.select_dtypes('number').columns

mean_imputer = SimpleImputer(strategy="mean")
scaler = StandardScaler()

#Numeric Pipeline
numeric_pipe = make_pipeline(mean_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

## ColumnTransformer

Create a tuple for each pipeline

In [22]:
#tuple= (name, pipeline, feature_list).
num_tuple = ('numeric', numeric_pipe, num_cols)
ohe_tuple = ('categorical', ohe_pipe, cat_cols)
ord_tuple = ('ordinal', ord_pipe, ord_cols)

Instantiate ColumnTransformer with all tuples.

In [23]:
col_transformer = ColumnTransformer(
    [num_tuple, ord_tuple, ohe_tuple],
    verbose_feature_names_out=False
)

Fit on training data only.

In [24]:
col_transformer.fit(X_train)



ColumnTransformer(transformers=[('numeric',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 Index(['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars',
       'potass', 'vitamins', 'weight', 'cups'],
      dtype='object')),
                                ('ordinal',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalenc...
                                                  OrdinalEncoder(categories=[['bottom',
                                                                              'middle',
                                                                              'top']])),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['shelf']),
                                ('categorical',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 Index(['name', 'mfr', 'type'], dtype='object'))],
                  verbose_feature_names_out=False)

Transform both training and test data.

In [29]:
X_train_processed = col_transformer.transform(X_train)
X_train_processed.head(40)

,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,weight,cups,shelf,name_100% Natural Bran,name_All-Bran,name_All-Bran with Extra Fiber,name_Apple Jacks,name_Basic 4,name_Bran Chex,name_Cheerios,name_Clusters,name_Cocoa Puffs,name_Corn Chex,name_Corn Flakes,name_Corn Pops,name_Cracklin' Oat Bran,name_Cream of Wheat (Quick),name_Crispix,name_Double Chex,name_Froot Loops,name_Frosted Flakes,name_Frosted Mini-Wheats,name_Fruit & Fibre Dates; Walnuts; and Oats,name_Fruity Pebbles,name_Golden Crisp,name_Golden Grahams,name_Grape Nuts Flakes,name_Honey Nut Cheerios,name_Honey-comb,name_Just Right Crunchy Nuggets,name_Just Right Fruit & Nut,name_Kix,name_Life,name_Lucky Charms,name_Maypo,name_Muesli Raisins; Dates; & Almonds,name_Mueslix Crispy Blend,name_Nut&Honey Crunch,name_Nutri-Grain Almond-Raisin,name_Oatmeal Raisin Crisp,name_Post Nat. Raisin Bran,name_Puffed Rice,name_Puffed Wheat,name_Quaker Oat Squares,name_Quaker Oatmeal,name_Raisin Nut Bran,name_Raisin Squares,name_Rice Chex,name_Rice Krispies,name_Shredded Wheat,name_Shredded Wheat 'n'Bran,name_Shredded Wheat spoon size,name_Special K,name_Total Corn Flakes,name_Total Raisin Bran,name_Total Whole Grain,name_Triples,name_Trix,name_Wheaties,name_Wheaties Honey Gold,mfr_A,mfr_G,mfr_K,mfr_MISSING,mfr_N,mfr_P,mfr_Q,mfr_R,type_C,type_H
30,-0.319703,-0.524507,-1.007451,-1.237705,-0.871334,-0.885939,1.992024,-0.751734,-0.128403,-0.173782,0.162657,-1.355719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
40,0.172812,-0.524507,0.040298,1.220654,-0.871334,1.343577,-0.795023,-0.751734,-0.128403,-0.173782,2.886968,-0.184871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39,1.650358,0.354813,0.040298,0.191573,-0.018050,1.120625,0.598501,0.031478,3.008288,1.660587,-0.408570,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
16,-0.319703,-0.524507,-1.007451,1.563681,-0.444692,1.343577,-1.027277,-0.822935,-0.128403,-0.173782,0.689943,-1.355719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
65,-0.812218,0.354813,-1.007451,-1.752245,0.408592,1.120625,-1.491785,0.387484,-1.173966,-0.173782,-0.760094,-1.355719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
54,-2.782279,-1.403826,-1.007451,-1.752245,-0.871334,-0.440036,-1.491785,-1.107739,-1.173966,-3.231064,0.689943,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
31,0.172812,-1.403826,0.040298,1.449339,-0.871334,0.005867,0.598501,-0.680533,-0.128403,-0.173782,-0.408570,-0.184871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0

In [28]:
X_test_processed = col_transformer.transform(X_test)
X_test_processed.head()

,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,weight,cups,shelf,name_100% Natural Bran,name_All-Bran,name_All-Bran with Extra Fiber,name_Apple Jacks,name_Basic 4,name_Bran Chex,name_Cheerios,name_Clusters,name_Cocoa Puffs,name_Corn Chex,name_Corn Flakes,name_Corn Pops,name_Cracklin' Oat Bran,name_Cream of Wheat (Quick),name_Crispix,name_Double Chex,name_Froot Loops,name_Frosted Flakes,name_Frosted Mini-Wheats,name_Fruit & Fibre Dates; Walnuts; and Oats,name_Fruity Pebbles,name_Golden Crisp,name_Golden Grahams,name_Grape Nuts Flakes,name_Honey Nut Cheerios,name_Honey-comb,name_Just Right Crunchy Nuggets,name_Just Right Fruit & Nut,name_Kix,name_Life,name_Lucky Charms,name_Maypo,name_Muesli Raisins; Dates; & Almonds,name_Mueslix Crispy Blend,name_Nut&Honey Crunch,name_Nutri-Grain Almond-Raisin,name_Oatmeal Raisin Crisp,name_Post Nat. Raisin Bran,name_Puffed Rice,name_Puffed Wheat,name_Quaker Oat Squares,name_Quaker Oatmeal,name_Raisin Nut Bran,name_Raisin Squares,name_Rice Chex,name_Rice Krispies,name_Shredded Wheat,name_Shredded Wheat 'n'Bran,name_Shredded Wheat spoon size,name_Special K,name_Total Corn Flakes,name_Total Raisin Bran,name_Total Whole Grain,name_Triples,name_Trix,name_Wheaties,name_Wheaties Honey Gold,mfr_A,mfr_G,mfr_K,mfr_MISSING,mfr_N,mfr_P,mfr_Q,mfr_R,type_C,type_H
4,0.0,-0.524507,1.088047,0.534600,-0.444692,-0.217084,0.366247,-1.335583,-0.128403,-0.173782,-0.408570,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
35,0.0,-1.403826,1.088047,0.763285,-0.444692,-0.662988,1.063009,-0.680533,-0.128403,-0.173782,0.689943,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
10,0.0,-1.403826,1.088047,0.763285,-0.871334,-0.662988,1.295263,-0.822935,-0.128403,-0.173782,-0.408570,-0.184871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
0,0.0,1.234133,0.040298,-0.265796,3.395084,-2.223649,-0.098261,2.665919,-0.128403,-0.173782,-2.254071,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
45,0.0,1.234133,2.135796,-0.037111,0.408592,0.228819,1.063009,1.099495,-0.128403,-0.173782,0.689943,0.985978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
